<a href="https://colab.research.google.com/github/Liao-HsienTing/PL-Repo./blob/main/114_1_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 區塊 1: 安裝所需套件
!pip install gspread google-auth-oauthlib pandas gradio

In [4]:
!pip uninstall -y gradio
!pip install gradio

Found existing installation: gradio 5.49.1
Uninstalling gradio-5.49.1:
  Successfully uninstalled gradio-5.49.1
  Using cached gradio-5.49.1-py3-none-any.whl.metadata (16 kB)
Using cached gradio-5.49.1-py3-none-any.whl (63.5 MB)


In [ ]:
# 區塊 1.1: 安裝 Google AI Python SDK
!pip install -q -U google-generativeai

ERROR: Operation cancelled by user
^C


In [1]:
# 區塊 2: 授權並連接到 Google Sheets
import gspread
from google.colab import auth
from google.auth import default
import pandas as pd

# 進行授權
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# --- 請填寫您的 Google Sheet 資訊 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1D6O9bWu97eWo0fls82iasAFiRQ75tYUk5Hi5Y6XVux8/edit?usp=sharing"
# ------------------------------------

try:
    # 開啟 Google Sheet 並選取第一個工作表
    worksheet = gc.open_by_url(SHEET_URL).sheet1
    print("✅ Google Sheet 連接成功！")
except gspread.exceptions.SpreadsheetNotFound:
    print("🛑 錯誤：找不到指定的 Google Sheet，請檢查連結是否正確。")
except Exception as e:
    print(f"🛑 發生未知錯誤：{e}")

✅ Google Sheet 連接成功！


In [6]:
# 區塊 3: 作業管理核心功能 (最終穩定版)
import json
from datetime import datetime
# 確保在檔案頂部有這些 import
import google.generativeai as genai
from google.colab import userdata
import pandas as pd
import requests  # Import requests here
from google.colab import userdata

# --- 欄位名稱設定 ---
COLUMNS = ["科目", "作業內容", "繳交日期", "狀態"]

# --- AI 分析功能 (最終修正版 - 使用 v1beta 和 gemini-flash-latest) ---
def get_ai_analysis(df):
    """
    【最終修正版】
    改用 v1beta 端點和 gemini-flash-latest 模型。
    """
    if df.empty:
        return "目前沒有任何作業紀錄，無法進行分析。"

    # 1. 準備 Prompt (這部分與您原本的程式碼相同)
    tasks_string = ""
    for index, row in df.iterrows():
        tasks_string += f"- 科目: {row['科目']}, 作業內容: {row['作業內容']}, 繳交日期: {row['繳交日期']}, 狀態: {row['狀態']}\n"

    prompt = f"""
    你是一位專業且有洞察力的學習分析師與時間管理教練。
    請根據以下這份學生的作業清單，提供一份全面、具體且有建設性的「學習分析報告」。

    學生的作業清單如下：
    {tasks_string}

    你的報告需要包含以下幾個部分，請使用 Markdown 格式化，並加上適當的表情符號，讓報告更生動易讀：

    1.  **🎯 總體任務概覽**
    2.  **💡 優先級建議**
    3.  **📈 潛在風險與提醒**
    4.  **💪 鼓勵與下一步**

    請直接生成這份報告。
    """

    # 2. 使用 Requests 呼叫 API
    try:
        api_key = userdata.get('week3_API')
        if not api_key:
            return "🛑 錯誤：無法在執行期間讀取到 Colab Secret 'week3_API'。"

        # --- 核心修正點：使用您提議的 v1beta 和 flash-latest 端點 ---
        url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-flash-latest:generateContent?key={api_key}"
        # -----------------------------------------------------------

        payload = {"contents": [{"parts": [{"text": prompt}]}]}
        headers = {'Content-Type': 'application/json'}

        response = requests.post(url, headers=headers, json=payload, timeout=30)

        # 檢查是否有 404 錯誤 (模型名稱錯誤)
        if response.status_code == 404:
            print(f"404 錯誤：找不到模型或端點。回應: {response.text}")
            return f"🛑 AI 分析失敗：404 找不到端點。請檢查模型名稱 'gemini-flash-latest' 是否拼寫正確，或嘗試 'gemini-1.5-flash-latest'。"

        # 檢查其他 HTTP 錯誤 (如 400, 403, 500)
        response.raise_for_status()
        result = response.json()

        try:
            generated_text = result['candidates'][0]['content']['parts'][0]['text']
            return generated_text
        except (KeyError, IndexError):
            print(f"從 API 回應中解析內容失敗。完整回應: {result}")
            return "🛑 AI 分析失敗：API 回應格式不正確。"

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP 錯誤發生：{http_err}")
        print(f"伺服器回應：{response.text}")
        # 特別處理 403 (權限) 錯誤
        if response.status_code == 403:
             return f"🛑 AI 分析失敗：{http_err}。請確認您已在 Cloud Console 中啟用 'Generative Language API' (或 'Vertex AI API')。"
        return f"🛑 AI 分析失敗：{http_err}。"
    except requests.exceptions.Timeout:
        return "🛑 AI 分析失敗：請求超時，請檢查網路連線。"
    except Exception as e:
        return f"🛑 AI 分析時發生未知錯誤：{e}"

# --- 以下是您專案的其他核心功能，保持不變 ---
def load_data_from_sheet():
    try:
        data = worksheet.get_all_records()
        if not data: return pd.DataFrame(columns=COLUMNS)
        df = pd.DataFrame(data)
        for col in COLUMNS:
            if col not in df.columns: df[col] = ""
        df['繳交日期'] = pd.to_datetime(df['繳交日期'], errors='coerce').dt.strftime('%Y-%m-%d') # Corrected date format here as well
        df = df.sort_values(by="繳交日期", ascending=True)
        return df
    except Exception as e: return pd.DataFrame(columns=COLUMNS)


def save_data_to_sheet(df):
    try:
        worksheet.clear()
        worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    except Exception as e: print(f"🛑 寫入資料時發生錯誤: {e}")

def save_analysis_to_sheet(analysis_text):
    try:
        try:
            analysis_worksheet = gc.open_by_url(SHEET_URL).worksheet("AI分析報告")
        except gspread.WorksheetNotFound:
            analysis_worksheet = gc.open_by_url(SHEET_URL).add_worksheet(title="AI分析報告", rows="100", cols="20")
        analysis_worksheet.clear()
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        analysis_worksheet.update('A1', f"學習分析報告 (產生時間: {current_time})")
        analysis_worksheet.update('A2', analysis_text)
        analysis_worksheet.format('A1', {'textFormat': {'bold': True, 'fontSize': 14}})
        return "✅ AI 分析報告已成功儲存至 Google Sheet！"
    except Exception as e: return f"🛑 儲存報告至 Google Sheet 時發生錯誤: {e}"

def add_task(subject, content, due_date, status):
    if not all([subject, content, due_date, status]): return "🛑 所有欄位都必須填寫！", load_data_from_sheet()
    try:
        due_date_str = pd.to_datetime(due_date).strftime('%Y-%m-%d')
        new_task = pd.DataFrame([[subject, content, due_date_str, status]], columns=COLUMNS)
        df = pd.concat([load_data_from_sheet(), new_task], ignore_index=True)
        save_data_to_sheet(df)
        return "✅ 任務新增成功！", df
    except Exception as e: return f"🛑 新增失敗：{e}", load_data_from_sheet()

def delete_task(task_identifier):
    if not task_identifier: return "🛑 請選擇要刪除的任務！", load_data_from_sheet(), gr.Dropdown(choices=[])
    df = load_data_from_sheet()
    mask = df.apply(lambda row: f"{row['科目']} - {row['作業內容']}", axis=1) != task_identifier
    if len(df[~mask]) == 0: return "🛑 找不到要刪除的任務。", df, gr.Dropdown(choices=get_task_choices())
    df_updated = df[mask]
    save_data_to_sheet(df_updated)
    return "✅ 任務刪除成功！", df_updated, gr.Dropdown(choices=get_task_choices(df_updated), value=None)

def query_tasks(status_filter, start_date, end_date):
    df = load_data_from_sheet()
    if status_filter != "全部": df = df[df["狀態"] == status_filter]
    if start_date and end_date:
        df['繳交日期'] = pd.to_datetime(df['繳交日期'])
        start = pd.to_datetime(start_date)
        end = pd.to_datetime(end_date)
        df = df[(df["繳交日期"] >= start) & (df["繳交日期"] <= end)]
        df['繳交日期'] = df['繳交日期'].dt.strftime('%Y-%m-%d')
    return df

def export_to_json(df):
    df.to_json('作業紀錄.json', orient='records', force_ascii=False, indent=4)
    return '作業紀錄.json'

def export_to_csv(df):
    df.to_csv('作業紀錄.csv', index=False, encoding='utf_8_sig')
    return '作業紀錄.csv'

def get_task_choices(df=None):
    if df is None: df = load_data_from_sheet()
    if df.empty: return []
    return df.apply(lambda row: f"{row['科目']} - {row['作業內容']}", axis=1).tolist()

In [ ]:
# 區塊 4: Gradio UI 介面 (AI 增強版)
import gradio as gr
import plotly.express as px
import json
from datetime import datetime
import requests  # Import requests here


# --- 欄位名稱設定 ---
COLUMNS = ["科目", "作業內容", "繳交日期", "狀態"]


# Define the functions that were causing the NameError
def load_data_from_sheet():
    try:
        # Assuming 'worksheet' is available from a previous cell's execution (z2btW4faJ_IO)
        # If not, you would need to re-open the sheet here.
        # For robustness within this cell, we could re-open the sheet:
        # gc = gspread.authorize(creds) # Assuming creds is available
        # worksheet = gc.open_by_url(SHEET_URL).sheet1 # Assuming SHEET_URL is available
        # However, relying on the previous cell's execution is the intended flow.
        # So, assuming worksheet is globally available after running cell z2btW4faJ_IO.
        data = worksheet.get_all_records()
        if not data: return pd.DataFrame(columns=COLUMNS)
        df = pd.DataFrame(data)
        for col in COLUMNS:
            if col not in df.columns: df[col] = ""
        df['繳交日期'] = pd.to_datetime(df['繳交日期'], errors='coerce').dt.strftime('%Y-%m-%d')
        df = df.sort_values(by="繳交日期", ascending=True)
        return df
    except Exception as e:
        print(f"🛑 讀取資料時發生錯誤: {e}")
        return pd.DataFrame(columns=COLUMNS)


def save_data_to_sheet(df):
    try:
        # Assuming 'worksheet' is available
        worksheet.clear()
        worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    except Exception as e: print(f"🛑 寫入資料時發生錯誤: {e}")

def save_analysis_to_sheet(analysis_text):
    try:
        # Assuming 'gc' and 'SHEET_URL' are available
        try:
            analysis_worksheet = gc.open_by_url(SHEET_URL).worksheet("AI分析報告")
        except gspread.WorksheetNotFound:
            analysis_worksheet = gc.open_by_url(SHEET_URL).add_worksheet(title="AI分析報告", rows="100", cols="20")
        analysis_worksheet.clear()
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        analysis_worksheet.update('A1', f"學習分析報告 (產生時間: {current_time})")
        analysis_worksheet.update('A2', analysis_text)
        analysis_worksheet.format('A1', {'textFormat': {'bold': True, 'fontSize': 14}})
        return "✅ AI 分析報告已成功儲存至 Google Sheet！"
    except Exception as e: return f"🛑 儲存報告至 Google Sheet 時發生錯誤: {e}"

def add_task(subject, content, due_date, status):
    if not all([subject, content, due_date, status]): return "🛑 所有欄位都必須填寫！", load_data_from_sheet()
    try:
        due_date_str = pd.to_datetime(due_date).strftime('%Y-%m-%d')
        new_task = pd.DataFrame([[subject, content, due_date_str, status]], columns=COLUMNS)
        df = pd.concat([load_data_from_sheet(), new_task], ignore_index=True)
        save_data_to_sheet(df)
        return "✅ 任務新增成功！", df
    except Exception as e: return f"🛑 新增失敗：{e}", load_data_from_sheet()

def delete_task(task_identifier):
    if not task_identifier: return "🛑 請選擇要刪除的任務！", load_data_from_sheet(), gr.Dropdown(choices=[])
    df = load_data_from_sheet()
    mask = df.apply(lambda row: f"{row['科目']} - {row['作業內容']}", axis=1) != task_identifier
    if len(df[~mask]) == 0: return "🛑 找不到要刪除的任務。", df, gr.Dropdown(choices=get_task_choices())
    df_updated = df[mask]
    save_data_to_sheet(df_updated)
    return "✅ 任務刪除成功！", df_updated, gr.Dropdown(choices=get_task_choices(df_updated), value=None)

def query_tasks(status_filter, start_date, end_date):
    df = load_data_from_sheet()
    if status_filter != "全部": df = df[df["狀態"] == status_filter]
    if start_date and end_date:
        df['繳交日期'] = pd.to_datetime(df['繳交日期'])
        start = pd.to_datetime(start_date)
        end = pd.to_datetime(end_date)
        df = df[(df["繳交日期"] >= start) & (df["繳交日期"] <= end)]
        df['繳交日期'] = df['繳交日期'].dt.strftime('%Y-%m-%d')
    return df

def export_to_json(df):
    df.to_json('作業紀錄.json', orient='records', force_ascii=False, indent=4)
    return '作業紀錄.json'

def export_to_csv(df):
    df.to_csv('作業紀錄.csv', index=False, encoding='utf_8_sig')
    return '作業紀錄.csv'

def get_task_choices(df=None):
    if df is None: df = load_data_from_sheet()
    if df.empty: return []
    return df.apply(lambda row: f"{row['科目']} - {row['作業內容']}", axis=1).tolist()


def create_plot(df):
    """建立狀態分佈的圓餅圖"""
    if df.empty or '狀態' not in df.columns or df['狀態'].isnull().all():
        return None
    status_counts = df['狀態'].value_counts().reset_index()
    status_counts.columns = ['狀態', '數量']
    fig = px.pie(status_counts, names='狀態', values='數量', title='作業狀態分佈', hole=.3)
    return fig

def update_ui_components():
    """一個統一的函數來更新所有UI組件"""
    df = load_data_from_sheet()
    plot = create_plot(df)
    choices = get_task_choices()
    return df, plot, gr.Dropdown(choices=choices, value=None)

# --- 新增的 AI 分析觸發函數 ---
def run_and_save_analysis():
    """執行 AI 分析並保存結果"""
    gr.Info("🤖 AI 正在為您分析，請稍候...")
    df = load_data_from_sheet()
    analysis_result = get_ai_analysis(df)
    save_status = save_analysis_to_sheet(analysis_result)
    return analysis_result, save_status

# --- Gradio 介面設計 ---
with gr.Blocks(theme=gr.themes.Soft(), title="AI 作業紀錄提醒系統") as app:
    gr.Markdown("# ✅ AI 作業紀錄提醒系統")
    gr.Markdown("一個整合了 Gemini AI 分析的智慧作業管理工具。")

    with gr.Row():
        with gr.Column(scale=2):
            gr.Label("目前作業總覽")
            output_df = gr.DataFrame(load_data_from_sheet, interactive=False)
            output_plot = gr.Plot(value=create_plot(load_data_from_sheet()))

        with gr.Column(scale=1):
            with gr.Tabs():
                # --- 新增 AI 分析分頁 ---
                with gr.TabItem("🤖 AI 智慧分析"):
                    gr.Markdown("點擊按鈕，讓 AI 為您分析目前的學習狀況、提供任務優先級建議，並找出潛在風險。")
                    ai_button = gr.Button("🚀 產生我的學習分析報告", variant="primary")
                    ai_output = gr.Markdown(label="AI 分析結果")
                    ai_save_status = gr.Textbox(label="儲存狀態", interactive=False)

                with gr.TabItem("📝 新增任務"):
                    add_subject = gr.Textbox(label="科目")
                    add_content = gr.Textbox(label="作業內容")
                    add_due_date = gr.Textbox(label="繳交日期", placeholder="格式: YYYY-MM-DD")
                    add_status = gr.Radio(["未完成", "已完成", "進行中"], label="狀態", value="未完成")
                    add_button = gr.Button("新增任務")
                    add_status_msg = gr.Textbox(label="執行狀態", interactive=False)

                with gr.TabItem("🗑️ 刪除任務"):
                    delete_dropdown = gr.Dropdown(choices=get_task_choices(), label="選擇要刪除的任務")
                    delete_button = gr.Button("確認刪除", variant="stop")
                    delete_status_msg = gr.Textbox(label="執行狀態", interactive=False)

                with gr.TabItem("🔍 查詢與匯出"):
                    gr.Markdown("### 條件查詢")
                    query_status = gr.Dropdown(["全部", "未完成", "已完成", "進行中"], label="依狀態查詢", value="全部")
                    query_start_date = gr.Textbox(label="開始日期", placeholder="格式: YYYY-MM-DD")
                    query_end_date = gr.Textbox(label="結束日期", placeholder="格式: YYYY-MM-DD")
                    query_button = gr.Button("執行查詢", variant="secondary")
                    gr.Markdown("---")
                    gr.Markdown("### 匯出資料")
                    json_button = gr.Button("匯出 JSON")
                    csv_button = gr.Button("匯出 CSV")
                    download_file = gr.File(label="下載檔案")

    # --- 按鈕事件綁定 ---
    # AI 分析按鈕
    ai_button.click(
        fn=run_and_save_analysis,
        inputs=[],
        outputs=[ai_output, ai_save_status]
    )

    # 新增按鈕
    add_button.click(fn=add_task, inputs=[add_subject, add_content, add_due_date, add_status], outputs=[add_status_msg, output_df]).then(fn=update_ui_components, inputs=[], outputs=[output_df, output_plot, delete_dropdown])

    # 刪除按鈕
    delete_button.click(fn=delete_task, inputs=[delete_dropdown], outputs=[delete_status_msg, output_df, delete_dropdown]).then(fn=update_ui_components, inputs=[], outputs=[output_df, output_plot, delete_dropdown])

    # 查詢按鈕
    query_button.click(fn=query_tasks, inputs=[query_status, query_start_date, query_end_date], outputs=[output_df]).then(fn=lambda df: create_plot(df), inputs=[output_df], outputs=[output_plot])

    # 匯出按鈕
    json_button.click(fn=export_to_json, inputs=[output_df], outputs=[download_file])
    csv_button.click(fn=export_to_csv, inputs=[output_df], outputs=[download_file])

# 啟動 Gradio 應用
app.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://436aaee13b8769f537.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
